# Naive Bayes Classifier

This is going to be the second classifier I´m going to use. 
The multinomial naive bayes classifier was part of lab 5 in our course. 


In [12]:
# import needed libaries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [3]:
# load the data
data = pd.read_csv("LabeledTweets.csv", ";")
data = data.drop(["Unnamed: 0", "index", "emotion_probability"], axis = 1)
data

,username,date,retweets,favorites,text,emotion
0,realDonaldTrump,09.02.20 00:47,13459,72445,A great coach and a fantastic guy. His endorse...,joy
1,realDonaldTrump,08.02.20 22:08,47880,215503,Pete Rose played Major League Baseball for 24 ...,sadness
2,realDonaldTrump,08.02.20 20:48,9452,37402,Total and complete Endorsement for Debbie Lesk...,joy
3,realDonaldTrump,08.02.20 20:40,17545,62484,Governor Cuomo wanted to see me this weekend. ...,sadness
4,realDonaldTrump,08.02.20 20:01,27437,120598,We will not be touching your Social Security o...,anger
...,...,...,...,...,...,...
86,realDonaldTrump,31.01.20 03:54,24418,100133,"This November, we are going to defeat the Radi...",joy
87,realDonaldTrump,31.01.20 03:48,19539,88388,"Thank you Iowa, I love you! https://www. pscp....",joy
88,realDonaldTrump,31.01.20 01:19,18878,73498,"Great poll in Iowa, where I just landed for a ...",joy
89,realDonaldTrump,30.01.20 23:04,22964,138237,Working closely with China and others on Coron...,joy


In [7]:
X = data.drop(["emotion", "username", "date", "retweets", "favorites"], axis = 1)
y = data["emotion"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

## Train classifier

In [38]:
# init the dictionary
dict_fit = {}

for index in range(X_train.shape[0]):
    
    # set up the pipeline

    # create pipeline
    Pipeline_CV_MNB = Pipeline([
        ('vect', CountVectorizer(lowercase=True)),
        ('clf', MultinomialNB()),
    ])
    
    # fit every mention
    # fit the pipeline with X (mention) & y (entity)
    fit = Pipeline_CV_MNB.fit(X = [X_train.iloc[index][0]],
                              y = [y_train.iloc[index]])
    
    dict_fit[index] = fit

In [48]:
for index in X_test.shape[0]:
    print(dict_fit.predict([X_test.iloc[index][0]]))

TypeError: 'int' object is not iterable

In [47]:
X_test.iloc[0][0]

'Don’t worry, it won’t happen! https:// twitter.com/TuckerCarlson/ status/1225607709044637697 …'

## Lab code

In [1]:
# function to create predictions
def prob6_func(df):
    # run every row of the df
    for row in range(df.shape[0]):
        # want to make predictions from df of problem 3
        # we gonna label the --NME-- of df_dev_small

        # we have to access the mention for each sentence in the df_dev_small
        sentence = df.iloc[row,1].split()

        # we need the beginning and the end to access the mention
        beg = df.iloc[row,2]
        end = df.iloc[row,3]
        mention = sentence[beg:end]

        # we also need to access the context, which is needed to make the prediction of the mention
        # 5 tokens to the left and the 5 tokens to the right of the mention
        context = sentence_test[max(0,(beg-5)):min((end+5), len(sentence))]
        # need the context as string for the prediction
        context_string = " ".join(context)

        ### ------ flag ------

        # create df, later check if mention exists
        mention_test_string = " ".join(mention)
        mention_df = df_contexts[df_contexts.mention == mention_test_string]

        # if df not empty -> take highest prob
        # else -- NME --
        if mention_df.shape[0] != 0:
            # make predictions
            label = dict_fit[mention_test_string].predict([context_string])[0]
            yield mention_test_string, label
        else:
            yield mention_test_string, "--NME--"